In [1]:
import shutil
import gzip
from pathlib import Path

import requests
import polars as pl
from tabulate import tabulate
from IPython.display import display, HTML

links = """name.basics.tsv.gz
title.akas.tsv.gz
title.basics.tsv.gz
title.crew.tsv.gz
title.episode.tsv.gz
title.principals.tsv.gz
title.ratings.tsv.gz""".split()

to_download = [f"https://datasets.imdbws.com/{l}" for l in links]
dest_dir = Path.home() / "data" / "imdb"

for fname, url in zip(links, to_download):
    dest = dest_dir / fname
    if dest.exists():
        continue

    dest.parent.mkdir(exist_ok=True, parents=True)

    response = requests.get(url, allow_redirects=True)
    response.raise_for_status()

    with dest.open(mode="wb") as f:  # replace with your desired filename and extension
        f.write(response.content)


for filename in dest_dir.glob("*.tsv.gz"):
    dest_file = dest_dir / filename.stem
    if dest_file.exists():
        continue

    with gzip.open(filename, "rb") as f_in, dest_file.open(mode="wb") as f_out:
        shutil.copyfileobj(f_in, f_out)

In [2]:
dest_joined = dest_dir / "imdb01-11-2024.parquet"

if not dest_joined.exists():
    csv_options = {
        "separator": "\t",
        "encoding": "utf8",
        "ignore_errors": True,
        "infer_schema_length": 10000,  # Adjust as needed
        "quote_char": None,  # Disable quoting to handle unescaped quotes
        'null_values': ['\\N'],
    }
    
    ratings = pl.scan_csv(dest_dir / "title.ratings.tsv", **csv_options)
    
    details = pl.scan_csv(dest_dir / "title.akas.tsv", **csv_options).select(
        ["titleId", "title", "region", "language"]
    )
    
    basics = pl.scan_csv(dest_dir / "title.basics.tsv", **csv_options).select(
        ["tconst", "startYear", "genres", "primaryTitle", "titleType"]
    )
    
    # Perform lazy joins
    joined = details.join(
        ratings,
        left_on="titleId",
        right_on="tconst",
        how="inner",  # Only keep matching rows
    ).join(basics, left_on="titleId", right_on="tconst", how="inner")

    joined.collect().write_parquet(dest_joined)
    joined = joined.unique()

joined = pl.scan_parquet(dest_joined)

In [3]:
# def show(df: pl.DataFrame):
#     return display(HTML(tabulate(df.to_dicts(), headers="keys", tablefmt="html")))

# pl.Config.set_tbl_rows(-1)


# # Apply filters using lazy evaluation
# good = joined.filter(
#     (pl.col("averageRating") >= 7.2)
#     & (pl.col("numVotes") > 50_000)
#     & (pl.col("startYear") > 2010)
#     & (pl.col("titleType").is_in(["movie", "tvMovie"]))
#     & (pl.col("region").is_null())
#     & (pl.col("genres").str.contains("Comedy"))
# ).unique(subset=["titleId"])

# result = good.select(
#     ["title", "primaryTitle", "startYear", "genres", "averageRating", "numVotes"]
# ).sort("averageRating", descending=True)

# final_df = result.collect()
# show(final_df)